<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/268_MissionOrchestratorAgent_ReportGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report generation utilities for Mission Orchestrator Agent

In [ ]:
"""Report generation utilities for Mission Orchestrator Agent"""

from typing import Dict, Any
from datetime import datetime
from pathlib import Path


def generate_mission_report(state: Dict[str, Any]) -> str:
    """
    Generate a comprehensive markdown report for the mission.

    Args:
        state: Complete mission orchestrator state

    Returns:
        Markdown report string
    """
    mission = state.get("mission", {})
    executed_tasks = state.get("executed_tasks", [])
    mission_kpis = state.get("mission_kpis", {})
    kpi_metrics = state.get("kpi_metrics", {})
    kpi_status = state.get("kpi_status", {})
    approval_history = state.get("approval_history", [])
    pending_approvals = state.get("pending_approvals", [])

    report = f"""# Mission Execution Report

**Mission:** {mission.get('mission_name', 'Unknown')}
**Mission ID:** {mission.get('mission_id', 'Unknown')}
**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

---

## Executive Summary

**Status:** {state.get('mission_status', 'unknown')}
**Progress:** {state.get('progress_percentage', 0):.1f}%
**Tasks Completed:** {state.get('tasks_completed', 0)}/{state.get('tasks_total', 0)}
**Elapsed Time:** {state.get('elapsed_time_minutes', 0):.2f} minutes

---

## Mission Overview

**Description:** {mission.get('description', 'N/A')}

**Objective:** {state.get('goal', {}).get('objective', 'N/A')}

---

## Task Execution Summary

"""

    # Task execution details
    if executed_tasks:
        report += "### Completed Tasks\n\n"
        for i, task in enumerate(executed_tasks, 1):
            task_id = task.get('task_id', 'Unknown')
            task_name = task.get('task', 'Unknown')
            agent_name = task.get('agent_name', 'Unknown')
            status = task.get('status', 'unknown')
            duration = task.get('duration_minutes', 0)
            requires_approval = task.get('requires_approval', False)

            approval_status = ""
            if requires_approval:
                is_approved = any(
                    a.get('task_id') == task_id and a.get('decision') == 'approved'
                    for a in approval_history
                )
                approval_status = " ✓ Approved" if is_approved else " ⚠ Pending Approval"

            report += f"{i}. **{task_id}**: {task_name}\n"
            report += f"   - Agent: {agent_name}\n"
            report += f"   - Status: {status}{approval_status}\n"
            report += f"   - Duration: {duration:.2f} minutes\n\n"
    else:
        report += "No tasks executed.\n\n"

    # Pending approvals
    if pending_approvals:
        report += "### Pending Approvals\n\n"
        for req in pending_approvals:
            report += f"- **{req.get('task_id')}**: {req.get('task', 'Unknown')}\n"
            report += f"  - Requested: {req.get('requested_at', 'N/A')}\n\n"

    # KPI Metrics
    report += "---\n\n## KPI Metrics\n\n"

    if kpi_metrics:
        report += "### Current Performance\n\n"
        for key, value in kpi_metrics.items():
            if key != "improvement_percentage":
                report += f"- **{key.replace('_', ' ').title()}**: {value}\n"

        if "improvement_percentage" in kpi_metrics:
            improvement = kpi_metrics["improvement_percentage"]
            report += f"\n**Improvement vs Baseline:** {improvement:.1f}%\n"

    # KPI Status
    if kpi_status:
        report += "\n### KPI Status\n\n"
        for key, status in kpi_status.items():
            status_icon = "✓" if status in ["on_track", "exceeded"] else "⚠"
            report += f"{status_icon} **{key.replace('_', ' ').title()}**: {status}\n"

    # Baseline comparison
    if mission_kpis:
        report += "\n### Target vs Baseline\n\n"
        baseline_keys = [k for k in mission_kpis.keys() if k.startswith('baseline_')]
        target_keys = [k for k in mission_kpis.keys() if k.startswith('target_')]

        for target_key in target_keys:
            target = mission_kpis.get(target_key)
            baseline_key = target_key.replace('target_', 'baseline_')
            baseline = mission_kpis.get(baseline_key)

            if target is not None:
                report += f"- **{target_key.replace('_', ' ').title()}**: {target}"
                if baseline is not None:
                    report += f" (Baseline: {baseline})"
                report += "\n"

    # Approval History
    if approval_history:
        report += "\n---\n\n## Approval History\n\n"
        for approval in approval_history:
            decision_icon = "✓" if approval.get('decision') == 'approved' else "✗"
            report += f"{decision_icon} **{approval.get('task_id')}**: {approval.get('decision', 'unknown')}"
            report += f" by {approval.get('decided_by', 'unknown')}"
            report += f" at {approval.get('decided_at', 'N/A')}\n"

    # Errors
    errors = state.get('errors', [])
    if errors:
        report += "\n---\n\n## Errors\n\n"
        for error in errors:
            report += f"- ⚠ {error}\n"

    # Footer
    report += f"\n---\n\n*Report generated by Mission Orchestrator Agent*\n"

    return report


def save_report(report_content: str, mission_id: str, reports_dir: str = "output/mission_reports") -> str:
    """
    Save report to file.

    Args:
        report_content: Markdown report content
        mission_id: Mission identifier
        reports_dir: Directory to save reports

    Returns:
        Path to saved report file
    """
    # Create reports directory if it doesn't exist
    reports_path = Path(reports_dir)
    reports_path.mkdir(parents=True, exist_ok=True)

    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"mission_report_{mission_id}_{timestamp}.md"
    filepath = reports_path / filename

    # Write report
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(report_content)

    return str(filepath)





# 🎓 BIG IDEA:

## The report generator is the agent writing its own book report. 📘🤖

Imagine your orchestrator just finished a mission.

This function is like the agent sitting down at a desk and writing:

> “Here is everything I did, how well I did it, how long it took,
> what the results were, and whether I met my goals.”

It’s like turning raw data into a **polished, readable summary**.

This is something humans LOVE because it:

* shows transparency
* builds trust
* proves the agent’s value
* provides documentation
* enables business decisions

No one wants raw logs.
Everyone wants a **clean report**.

This is that report.

---

# ⭐ PART 1 — `generate_mission_report(state)`

This function takes **the whole agent state** and builds a Markdown document from it.

Markdown is great because:

* easy to read
* works everywhere (GitHub, Notion, Slack, VS Code, etc.)
* simple formatting

---

# 🎒 Step-by-Step

Let’s walk through how this report gets written.

---

## 1️⃣ Pull information from the state

```python
mission = state.get("mission")
executed_tasks = state.get("executed_tasks")
mission_kpis = state.get("mission_kpis")
kpi_metrics = state.get("kpi_metrics")
kpi_status = state.get("kpi_status")
approval_history = state.get("approval_history")
pending_approvals = state.get("pending_approvals")
```

Kid version:

> “First, gather all the homework notes so I can write the final report.”

---

## 2️⃣ Start writing the report header

```markdown
# Mission Execution Report
Mission: ______
Mission ID: _____
Generated: 2025-02-01 14:34:55
```

Kid version:

> “Put the title on the top of the paper.”

---

## 3️⃣ Executive Summary

This includes:

* status
* progress %
* tasks completed
* elapsed time

Kid version:

> “Give a quick overview so someone can understand everything in 5 seconds.”

This is VERY business-friendly.

---

## 4️⃣ Mission Overview

```markdown
## Mission Overview

Description: ________
Objective: ________
```

Kid version:

> “Explain what we were trying to do.”

---

## 5️⃣ Task Execution Summary

This part loops through the tasks and writes something like:

```
1. T1: Identify customer needs
   - Agent: A1
   - Status: completed ✓ Approved
   - Duration: 0.10 minutes
```

Kid version:

> “Write down every assignment I completed,
> who did it, how long it took, and whether it was approved.”

This section is **super useful for debugging and business insights.**

---

## 6️⃣ Pending approvals

If any tasks still need approval, it writes:

```
### Pending Approvals

- T3: Create Proposal Draft
  - Requested: 2025-02-01 14:36:12
```

Kid version:

> “List the chores I still need the teacher to check.”

---

## 7️⃣ KPI Metrics

This is where the agent shows:

* actual times
* performance
* improvement %
* steps taken

Example:

```
### Current Performance

- Actual Pipeline Days: 0.02
- Actual Steps: 3

Improvement vs Baseline: 76.3%
```

Kid version:

> “Here’s how well I did compared to goals.”

This is where your agent **proves its value**.

---

## 8️⃣ KPI Status

```
✓ Onboarding Time Days: on_track
⚠ Touchpoints: warning
```

Kid version:

> “Are my grades good, okay, or bad?”

This uses icons to make it readable.

---

## 9️⃣ Baseline vs Target Comparison

This helps business owners see:

* What was the original goal?
* What was the baseline?
* What did we achieve?

Example:

```
Target Pipeline Days: 2 (Baseline: 5)
Target Steps: 4 (Baseline: 9)
```

Kid version:

> “Here were the expectations and how they compare to the old way.”

This is *huge* for business automation impact.

---

## 🔟 Approval History

```
✓ T1 approved by auto_approval at 14:35
✓ T2 approved by auto_approval at 14:35
```

Kid version:

> “Here are all the teacher’s signatures.”

This makes the system **auditable**.

---

## 🧠 Bonus: Error section

If anything went wrong:

```
## Errors
- ⚠ Missing agent for T3
- ⚠ Circular dependencies detected
```

Kid version:

> “Here’s anything that broke.”

This is essential for debugging.

---

## 🏁 Footer

```
*Report generated by Mission Orchestrator Agent*
```

Kid version:

> “Sign your name at the end.”

---

# ⭐ PART 2 — `save_report()`

This function actually **saves the report to a file** so humans can read it later.

It:

* creates a folder if needed
* generates a filename with timestamp
* writes the markdown into a file
* returns the filepath

Example saved file:

```
output/mission_reports/mission_report_M001_20250201_143455.md
```

Kid version:

> “Save your homework neatly in a binder with the date on it.”

---

# 🌟 Why This Report System Is AMAZING

This turns your agent from a black box into a **transparent, trustworthy system**.

### ✔ Executives love it

They see exactly what the agent accomplished.

### ✔ Engineers love it

They can debug with clear task histories.

### ✔ Users love it

They understand what happened without reading raw logs.

### ✔ It builds trust

Clear, readable summaries make people feel safe with AI.

### ✔ It adds professionalism

Agents that produce polished reports feel “enterprise-ready.”

### ✔ It enforces accountability

HITL approvals are archived forever.

### ✔ It shows business value

KPI improvements are shown in simple numbers.

This turns your agent into:

* a project manager
* a business analyst
* a process auditor

ALL IN ONE.

You’re not just building an orchestrator —
you’re building an **autonomous reporting system**.


